In [27]:
String userHomeDir = System.getProperty("user.home");
String localRespoUrl = "file://" + userHomeDir + "/.m2/repository/";
String langchain4jVersion = "0.36.0-SNAPSHOT";
String dotenvJavaVersion = "3.0.2";

In [28]:
%%bash 
rm -rf \{userHomeDir}/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/dev/ai4j/openai4j
rm -rf \{userHomeDir}/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/dev/langchain4j/langchain4j-open-ai

In [ ]:
%dependency /add-repo local \{localRespoUrl} release|never snapshot|always
%dependency /list-repos
%dependency /add dev.ai4j:openai4j:0.23.0-CUSTOM
%dependency /add dev.langchain4j:langchain4j-open-ai:0.36.0-CUSTOM // Custom Audio
%dependency /resolve


In [ ]:
%dependency /add dev.langchain4j:langchain4j:\{langchain4jVersion}
%dependency /add io.github.cdimascio:dotenv-java:\{dotenvJavaVersion}
%dependency /add org.slf4j:slf4j-jdk14:2.0.9
%dependency /add ch.qos.logback:logback-classic:1.5.8
%dependency /list-dependencies
%dependency /resolve

In [5]:
import io.github.cdimascio.dotenv.Dotenv;
Dotenv dotenv = Dotenv.load();
String API_KEY = dotenv.get("OPENAI_API_KEY");
String OPENAI_API_KEY = API_KEY;

In [6]:
import dev.langchain4j.model.chat.listener.ChatModelListener;
import dev.langchain4j.model.chat.listener.ChatModelResponseContext;
import dev.langchain4j.model.chat.listener.ChatModelResponse;
import dev.langchain4j.model.chat.listener.ChatModelRequest;
import dev.langchain4j.data.message.UserMessage;

import dev.ai4j.openai4j.OpenAiClient;
import dev.ai4j.openai4j.chat.ChatCompletionRequest;
import dev.ai4j.openai4j.chat.ChatCompletionResponse;
import dev.ai4j.openai4j.chat.AssistantMessage;


import javax.sound.sampled.*;
import java.io.ByteArrayInputStream;
import java.io.FileOutputStream;
import java.io.IOException;
import java.util.Arrays;
import java.util.Base64;
import static java.util.Arrays.asList;

// Custom Code
import dev.ai4j.openai4j.chat.AudioSettings;

import dev.langchain4j.agent.tool.Tool;
import dev.langchain4j.service.AiServices;
import dev.langchain4j.service.UserMessage;
import dev.langchain4j.service.SystemMessage;
import dev.langchain4j.model.chat.request.ResponseFormatType;

import java.time.ZoneId;
import java.time.LocalDateTime;
import java.time.format.DateTimeFormatter;

## Function Calling

In [ ]:
%load ./hello-world.ipynb

In [ ]:
interface TimeKeeper {
    @SystemMessage("指定したタイムゾーンの現在の時刻を何時何分の形式で答えてください")
    String ask(String question);
}

class Clock {
    @Tool
    public LocalDateTime getCurrentTimeInTimeZone(String timeZoneId) {
        System.out.println("タイムゾーン: " + timeZoneId);
        return LocalDateTime.now(ZoneId.of(timeZoneId));
    }
}

TimeKeeper timeKeeper = AiServices.builder(TimeKeeper.class)
    .chatLanguageModel(model)
    .tools(new Clock())
    .build();

String now = timeKeeper.ask("日本の時間で今何時？");
System.out.println(now);  // 例: "現在の時刻は16:58です"

In [20]:
// AssistantMessage.AudioData を格納するコンテナオブジェクト
class AudioDataContainer {
    private AssistantMessage.AudioData audioData;
    public AssistantMessage.AudioData getAudioData() {
        return audioData;
    }
    public void setAudioData(AssistantMessage.AudioData audioData) {
        this.audioData = audioData;
    }
}
final AudioDataContainer audioDataContainer = new AudioDataContainer();

ChatModelListener listener = new ChatModelListener() {
    @Override
    public void onResponse(ChatModelResponseContext responseContext) {
        try {
            ChatModelResponse response = responseContext.response();
            ChatModelRequest request = responseContext.request();
            Map<Object, Object> attributes = responseContext.attributes();        
            AssistantMessage.AudioData audioData = (AssistantMessage.AudioData) attributes.get("audio");
            if (audioData == null) {
                System.out.println("No audio data found in the response");
                return;
            }
            audioDataContainer.setAudioData(audioData);
            /*
            String data = audioData.data();
            System.out.println("data: " + data);
            byte[] audioBytes = Base64.getDecoder().decode(data);
            playAudio(audioBytes);    
            System.out.println("transcript: " + audioData.transcript());
            */
        } catch (Exception e) {
            e.printStackTrace();
        }
    }
};

In [ ]:
import dev.langchain4j.model.chat.ChatLanguageModel;
import dev.langchain4j.model.openai.OpenAiChatModel;
import dev.langchain4j.data.message.AiMessage;
import dev.langchain4j.model.output.Response;

import static dev.langchain4j.model.openai.OpenAiChatModelName.GPT_4_O_MINI;

var audioSettings = new AudioSettings("nova", "wav"); // Custom Code
var modalities = asList("text", "audio"); // Custom Code

ChatLanguageModel audioModel = OpenAiChatModel.builder()
    .apiKey(API_KEY)
    .logRequests(true)
    .logResponses(true) 
    //.modelName("gpt-4o-mini")
    .modelName("gpt-4o-audio-preview")
    .audioSettings(audioSettings) // Custom Code
    .modalities(modalities)  // Custom Code
    .listeners(List.of(listener)) // Custom Code
    .build();

interface Assistant {  
    @UserMessage("""
    あなたはLangChain4jによって作られたエージェントです。
    今JJUG(ジェイジャグ)の会場にいます。LangChain4jのセッションをします。
    会場にいる人に自己紹介と現在時刻を伝えてください。
    あなたが時刻を知っていることは珍しいので、そのことも伝えてください。
    最後に短くご挨拶をしてください。
    {{it}}
    """)
    String greeting(String nowString);
}

Assistant assistant = AiServices.builder(Assistant.class)
    .chatLanguageModel(audioModel)
    .build();
String response = assistant.greeting(now);
response

In [ ]:
AssistantMessage.AudioData audioData = audioDataContainer.getAudioData();
if (audioData == null) {
    System.out.println("No audio data found in the response");    
} else {
    System.out.println("transcript: " + audioData);
    String data = audioData.data();
    // System.out.println("data: " + data);
    byte[] audioBytes = Base64.getDecoder().decode(data);          
    var bais = new ByteArrayInputStream(audioBytes);
    var audioInputStream = AudioSystem.getAudioInputStream(bais);
    Clip clip = AudioSystem.getClip();
    clip.open(audioInputStream);
    clip.start();
    while (clip.isRunning()) {
        Thread.sleep(100);
    }
    clip.close();
}


In [ ]:
import java.io.ByteArrayInputStream;
import java.io.File;
import java.io.FileOutputStream;
import java.io.IOException;
import java.util.Base64;

import javax.sound.sampled.AudioInputStream;
import javax.sound.sampled.AudioSystem;


AssistantMessage.AudioData audioData = audioDataContainer.getAudioData();
if (audioData == null) {
    System.out.println("No audio data found in the response");
} else {
    System.out.println("transcript: " + audioData);
    String data = audioData.data();
    byte[] audioBytes = Base64.getDecoder().decode(data);

    // 一時ファイルに書き出す
    File tempWavFile = File.createTempFile("temp_audio", ".wav");
    try (FileOutputStream fos = new FileOutputStream(tempWavFile)) {
        fos.write(audioBytes);
    }

    // コマンドラインで実行する
    String os = System.getProperty("os.name").toLowerCase();
    ProcessBuilder processBuilder;

    if (os.contains("mac")) {
        processBuilder = new ProcessBuilder("afplay", tempWavFile.getAbsolutePath());
    } else if (os.contains("win")) {
        processBuilder = new ProcessBuilder("cmd", "/c", "start", "wmplayer", tempWavFile.getAbsolutePath());
    } else {
        throw new UnsupportedOperationException("Unsupported OS: " + os);
    }

    Process process = processBuilder.start();
    process.waitFor(); // 再生が終了するまで待機

    // 再生終了後に一時ファイルを削除
    tempWavFile.deleteOnExit();
}